# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


## Задачи для совместного разбора

1. Рассмотрите, как можно выстраивать конвейер предобработки данных при помощи `Pipeline` из `sklearn`

In [ ]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [ ]:
X, y = make_regression(
    n_samples=1000,
    n_features=5
)

In [ ]:
X.shape

(1000, 5)

In [ ]:
pipe = Pipeline(
    [
        ("scaling", MinMaxScaler()),
        ("poly", PolynomialFeatures(
        )),
        ("lr", LinearRegression())
    ]
).fit(X, y)

In [ ]:
#pipe.predict(X)

2. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [ ]:
from torch.utils.data import Dataset, DataLoader #random_split

In [ ]:
class SyntDataset(Dataset):
  def __init__(self, transform: callable = None, **make_regression_args):
    self.X, self.y = make_regression(**make_regression_args)
    self.transform = transform

  def __getitem__(self, idx):
    x = self.X[idx]
    if self.transform is not None:
      x = self.transform(x)
    return x, self.y[idx]
    #return {"x":self.X[idx], "y":self.y[idx]}

  def __len__(self):
    return len(self.X)

In [ ]:
import numpy as np
def add_squares(x):
  return np.c_[x, x**2]

In [ ]:
d = SyntDataset(transform=add_squares, n_samples=1000, n_features=5)
d[0:2]

(array([[ 0.95019763, -0.09542377,  0.28638505,  1.77263569, -1.57771585,
          0.90287553,  0.0091057 ,  0.08201639,  3.1422373 ,  2.48918731],
        [ 0.33672097, -0.07940222, -0.1255191 , -0.16217775,  0.41135245,
          0.11338101,  0.00630471,  0.01575505,  0.02630162,  0.16921084]]),
 array([20.52483792,  5.10495385]))

In [ ]:
class SquareN:
  def __init__(self, n):
    self.n = n

  def __call__(self, x):
    for _ in range(self.n):
      x = add_squares(x)
    return x

In [ ]:
o = SquareN(2)
#o(d.X[:4])
#add_squares(d.X[:4])

In [ ]:
d = SyntDataset(transform=SquareN(2), n_samples=1000, n_features=5)
#d[0:2]

In [ ]:
dl = DataLoader(d, batch_size=32)

In [ ]:
for (batch_X, batch_y) in dl:
  print(batch_X.shape, batch_y.shape)
  break

torch.Size([32, 5, 4]) torch.Size([32])


## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`. Используя `Pipeline` из `sklearn`, закодируйте значения в нечисловых столбцах целыми числами, после чего нормализуйте получившиеся признаки. Выведите преобразованные данные на экран.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv("bank-full.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [ ]:
encoder_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
numeric_cols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'day']
columns = encoder_cols.extend(numeric_cols)

In [ ]:
#columns = df.columns
encoder_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
numeric_cols = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous', 'day']
cols = encoder_cols + numeric_cols
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', OrdinalEncoder(), encoder_cols),
        ('scaler', StandardScaler(), numeric_cols)])

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

transformed_data = pd.DataFrame(pipeline.fit_transform(df), columns=cols)
transformed_data.head()

,job,marital,education,default,housing,loan,contact,month,poutcome,y,age,balance,duration,campaign,pdays,previous,day
0,4.0,1.0,2.0,0.0,1.0,0.0,2.0,8.0,3.0,0.0,1.606965,0.256419,0.011016,-0.569351,-0.411453,-0.25194,-1.298476
1,9.0,2.0,1.0,0.0,1.0,0.0,2.0,8.0,3.0,0.0,0.288529,-0.437895,-0.416127,-0.569351,-0.411453,-0.25194,-1.298476
2,2.0,1.0,1.0,0.0,1.0,1.0,2.0,8.0,3.0,0.0,-0.747384,-0.446762,-0.707361,-0.569351,-0.411453,-0.25194,-1.298476
3,1.0,1.0,3.0,0.0,1.0,0.0,2.0,8.0,3.0,0.0,0.571051,0.047205,-0.645231,-0.569351,-0.411453,-0.25194,-1.298476
4,11.0,2.0,3.0,0.0,0.0,0.0,2.0,8.0,3.0,0.0,-0.747384,-0.447091,-0.233620,-0.569351,-0.411453,-0.25194,-1.298476


<p class="task" id="2"></p>

2\. Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` и `i`-ую строку из `obj.y`.
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        self.y = data["y"]
        self.X = data.drop(columns=["y"])

    def __getitem__(self, idx) -> tuple:
      return self.X.iloc[idx].values, self.y[idx]

    def __len__(self) -> int:
        return len(self.X)

In [ ]:
obj = BankDatasetBase(transformed_data)

In [ ]:
obj[:2]

(array([[ 4.        ,  1.        ,  2.        ,  0.        ,  1.        ,
          0.        ,  2.        ,  8.        ,  3.        ,  1.60696496,
          0.25641925,  0.0110161 , -0.56935064, -0.41145311, -0.25194037,
         -1.29847633],
        [ 9.        ,  2.        ,  1.        ,  0.        ,  1.        ,
          0.        ,  2.        ,  8.        ,  3.        ,  0.28852927,
         -0.43789469, -0.41612696, -0.56935064, -0.41145311, -0.25194037,
         -1.29847633]]),
 0    0.0
 1    0.0
 Name: y, dtype: float64)

In [ ]:
len(obj)

45211

<p class="task" id="3"></p>

3\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательный аргумент `transform: callable`;
* если данный аргумент был передан, то при получении `i`-го элемента, нужно вызвать `transform(x, y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность.

In [ ]:
class BankDataset(Dataset):
  def __init__(self, data: pd.DataFrame, transform: callable = None) -> None:
      self.y = data["y"]
      self.X = data.drop(columns=["y"])
      self.transform = transform

  def __getitem__(self, idx) -> tuple:
    X_, y_ = self.X.iloc[idx], self.y[idx]
    if self.transform is not None:
      X_, y_ = self.transform(X_, y_)
    return X_, y_

  def __len__(self) -> int:
      return len(self.X)

In [ ]:
def transform(x, y):
  return x ** 2, y + 1 if y == 0 else y - 1

In [ ]:

obj2 = BankDataset(transformed_data, transform=transform)
obj2[1]

(job          81.000000
 marital       4.000000
 education     1.000000
 default       0.000000
 housing       1.000000
 loan          0.000000
 contact       4.000000
 month        64.000000
 poutcome      9.000000
 age           0.083249
 balance       0.191752
 duration      0.173162
 campaign      0.324160
 pdays         0.169294
 previous      0.063474
 day           1.686041
 Name: 1, dtype: float64,
 1.0)

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет два параметра (признаки и класс) и возвращает кортеж, состоящих двух элементов: набора признаков, в котором нечисловые характеристики закодированы целыми числами; и чисел - меток классов;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `LabelEncoderTransform`.

In [ ]:
import numpy as np

In [ ]:
encoder_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

In [ ]:
class LabelEncoderTransform:
    def __init__(self, category_columns: list[str]) -> None:
        self.category_columns = category_columns
        self.columns = {}
        self.target = {}
        self.k = -1

    def getdic(self, x, col):
      if col not in self.columns:
        encoders = {}
        self.columns[col] = encoders
      if x not in self.columns[col].keys():
        j = max(self.columns[col].values()) + 1 if self.columns[col] != {} else 0
        self.columns[col][x] = j
      return self.columns[col][x]

    def __call__(self, x, y) -> tuple:
      X = x.copy()
      Y = []
      if type(x) == pd.Series:
        for col in self.category_columns:
          if col != 'y':
            X[col] = self.getdic(X[col], col)
      else:
        for col in self.category_columns:
          if col != 'y':
            X[col] = X[col].map(lambda l: self.getdic(l, col))
      if type(y) == str:
        if y not in self.target.keys():
          self.target[y] = self.k + 1
          self.k += 1
        Y = self.target[y]
      else:
        for el in y:
          if el not in self.target.keys():
            self.target[el] = self.k + 1
            self.k += 1
          Y.append(self.target[el])
      return X.astype(np.float64), np.array(Y).astype(np.float64)

In [ ]:
obj3 = BankDataset(df, LabelEncoderTransform(encoder_cols))

In [ ]:
obj3[0]

(age            58.0
 job             0.0
 marital         0.0
 education       0.0
 default         0.0
 balance      2143.0
 housing         0.0
 loan            0.0
 contact         0.0
 day             5.0
 month           0.0
 duration      261.0
 campaign        1.0
 pdays          -1.0
 previous        0.0
 poutcome        0.0
 Name: 0, dtype: float64,
 array(0.))

In [ ]:
obj3[:3]

(    age  job  marital  education  default  balance  housing  loan  contact  \
 0  58.0  0.0      0.0        0.0      0.0   2143.0      0.0   0.0      0.0   
 1  44.0  1.0      1.0        1.0      0.0     29.0      0.0   0.0      0.0   
 2  33.0  2.0      0.0        1.0      0.0      2.0      0.0   1.0      0.0   
 
    day  month  duration  campaign  pdays  previous  poutcome  
 0  5.0    0.0     261.0       1.0   -1.0       0.0       0.0  
 1  5.0    0.0     151.0       1.0   -1.0       0.0       0.0  
 2  5.0    0.0      76.0       1.0   -1.0       0.0       0.0  ,
 array([0., 0., 0.]))

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и возвращает кортеж, состоящих 2 тензоров.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании список из объектов LabelEncoderTransform и ToTensor.

In [ ]:
import torch as th

In [ ]:
class ToTensor(object):
    def __call__(self, X, y) -> tuple:
        return th.Tensor(X.values), th.Tensor(y)

class Compose(object):
    def __init__(self, transforms: list) -> None:
        self.transforms = transforms

    def __call__(self, X, y):
        for transform in self.transforms:
          X, y = transform(X, y)
        return X, y

In [ ]:
obj4 = BankDataset(df, Compose([LabelEncoderTransform(encoder_cols), ToTensor()]))

In [ ]:
obj4[:3]

(tensor([[ 5.8000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           2.1430e+03,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  2.6100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 4.4000e+01,  1.0000e+00,  1.0000e+00,  1.0000e+00,  0.0000e+00,
           2.9000e+01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  1.5100e+02,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00],
         [ 3.3000e+01,  2.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           2.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  5.0000e+00,
           0.0000e+00,  7.6000e+01,  1.0000e+00, -1.0000e+00,  0.0000e+00,
           0.0000e+00]]),
 tensor([0., 0., 0.]))

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета.

In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
train, test = random_split(obj4, [0.75, 0.25])

In [ ]:
train[0]

(tensor([ 32.,   3.,   2.,   1.,   0., 130.,   0.,   0.,   1.,  30.,   1., 194.,
           3.,  -1.,   0.,   0.]),
 tensor(0.))

In [ ]:
dl = DataLoader(train, batch_size=64)

In [ ]:
for (train_X, train_y) in dl:
  print(train_X.shape, train_y[0])
  break

torch.Size([64, 16]) tensor(0.)


In [ ]:
def custom_collate_fn(batch):
  tensors, targets = zip(*batch)
  tensors = th.stack(tensors).reshape((64, 2, 8))
  return tensors, th.Tensor(targets)

In [ ]:
train_dataloader = DataLoader(train, batch_size=64, collate_fn=custom_collate_fn, shuffle=True)

In [ ]:
for (train_X, train_y) in train_dataloader:
  print(train_X.shape, train_y.shape)
  break

torch.Size([64, 2, 8]) torch.Size([64])


## Обратная связь
- [x] Хочу получить обратную связь по решению